<a href="https://colab.research.google.com/github/DhanushDhanasekaran/NullClass-Data-Analysis/blob/main/Tweet_Interaction_Analysis_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Step 0: Install required packages if not already installed
# pip install datasets pandas matplotlib seaborn pytz

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import datetime
import pytz
from datasets import load_dataset

# -----------------------------
# Step 1: Load a free Twitter dataset from Hugging Face
# -----------------------------
dataset = load_dataset("tweet_eval", "sentiment", split="train")
df = pd.DataFrame(dataset)
df.rename(columns={"text": "tweet_text"}, inplace=True)

# -----------------------------
# Step 2: Simulate tweet_datetime and interactions
# -----------------------------
def random_datetime_even_day(start_hour=15, end_hour=17):
    day = random.choice([2,4,6,8,10,12,14,16,18,20])
    month = 9
    year = 2025
    hour = random.randint(start_hour, end_hour-1)
    minute = random.randint(0,59)
    second = random.randint(0,59)
    return datetime(year, month, day, hour, minute, second)

df['tweet_datetime'] = [random_datetime_even_day() for _ in range(len(df))]
df['media'] = [random.choice([0,1]) for _ in range(len(df))]
df['url_clicks'] = [random.randint(0, 20) for _ in range(len(df))]
df['profile_clicks'] = [random.randint(0, 20) for _ in range(len(df))]
df['hashtag_clicks'] = [random.randint(0, 20) for _ in range(len(df))]
df.loc[(df['url_clicks']==0) & (df['profile_clicks']==0) & (df['hashtag_clicks']==0), 'url_clicks'] = 1

# -----------------------------
# Step 3: Filter tweets according to homework instructions
# -----------------------------
df['hour'] = df['tweet_datetime'].dt.hour
df_filtered = df[(df['hour'] >= 15) & (df['hour'] < 17)]
df_filtered = df_filtered[df_filtered['tweet_datetime'].dt.day % 2 == 0]
df_filtered = df_filtered[df_filtered['tweet_text'].str.split().str.len() > 40]
df_filtered = df_filtered[(df_filtered['url_clicks']>0) | (df_filtered['profile_clicks']>0) | (df_filtered['hashtag_clicks']>0)]

# -----------------------------
# Step 4: Categorize tweets
# -----------------------------
def categorize_tweet(row):
    if row['media']==1:
        return 'Media'
    elif row['url_clicks']>0:
        return 'Link'
    elif row['hashtag_clicks']>0:
        return 'Hashtag'
    else:
        return 'Other'

df_filtered['tweet_category'] = df_filtered.apply(categorize_tweet, axis=1)

# -----------------------------
# Step 5: Aggregate interactions
# -----------------------------
agg_df = df_filtered.groupby('tweet_category')[['url_clicks','profile_clicks','hashtag_clicks']].sum().reset_index()

# -----------------------------
# Step 6: Dashboard logic – only show chart 3PM-5PM IST
# -----------------------------
ist = pytz.timezone('Asia/Kolkata')
current_hour_ist = datetime.now(ist).hour

if 15 <= current_hour_ist < 17:
    plt.figure(figsize=(10,6))
    agg_df_melted = agg_df.melt(id_vars='tweet_category', var_name='interaction_type', value_name='sum_clicks')
    sns.barplot(x='tweet_category', y='sum_clicks', hue='interaction_type', data=agg_df_melted)
    plt.title("Sum of Interactions by Tweet Category (3PM-5PM IST, Even Dates, Word Count>40)")
    plt.xlabel("Tweet Category")
    plt.ylabel("Sum of Clicks")
    plt.legend(title='Interaction Type')
    plt.tight_layout()
    plt.show()
else:
    print("Chart will only display between 3 PM – 5 PM IST.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sentiment/train-00000-of-00001.parquet:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

sentiment/test-00000-of-00001.parquet:   0%|          | 0.00/901k [00:00<?, ?B/s]

sentiment/validation-00000-of-00001.parq(…):   0%|          | 0.00/167k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Chart will only display between 3 PM – 5 PM IST.
